<a href="https://colab.research.google.com/github/Akashsindhu/stackoverflow-tags-classification/blob/master/live_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
% cd My\ Drive/Colab Notebooks
# % cd Colab
df = pd.read_csv('SO_ml_tags_avocado_188k_v2.csv')

/gdrive/My Drive/Colab Notebooks


In [0]:
df.head()

,extracted_tags,original_tags,text
0,"matplotlib,pandas","python,matplotlib,pandas",setting xticks and yticks for scatter plot mat...
1,"scikitlearn,keras","python,numpy,scikit-learn,keras,grid-search",gridseachcv - valueerror: found input variable...
2,"matplotlib,scikitlearn","python,numpy,matplotlib,scikit-learn,nmf",non negative matrix factorisation in python on...
3,"pandas,tensorflow","python,pandas,tensorflow,time-series",avocado equivalent to avocado.dataframe.resamp...
4,"matplotlib,pandas","python,matplotlib,plot,pandas",how to plot on avocado python i have a data fr...


In [0]:
df = df.drop('original_tags', axis =1)

In [0]:
df.head()

,extracted_tags,text
0,"matplotlib,pandas",setting xticks and yticks for scatter plot mat...
1,"scikitlearn,keras",gridseachcv - valueerror: found input variable...
2,"matplotlib,scikitlearn",non negative matrix factorisation in python on...
3,"pandas,tensorflow",avocado equivalent to avocado.dataframe.resamp...
4,"matplotlib,pandas",how to plot on avocado python i have a data fr...


In [0]:
df.isna().sum()

extracted_tags    0
text              0
dtype: int64

In [0]:
seed = 7
np.random.seed(seed)
df = shuffle(df, random_state=seed)


In [0]:
df.head()

,extracted_tags,text
17449,pandas,error when adding a new column to avocado data...
111300,matplotlib,how to add counts of points as a label in a sp...
120503,matplotlib,top and bottom line on errorbar with python an...
168430,scikitlearn,avocado-learn using gridsearchcv on decisiontr...
108052,matplotlib,how to render avocado window in template page(...


In [0]:
train_size = int(len(df) * .8)

train_qs = df['text'].values[:train_size]
test_qs = df['text'].values[train_size:]
# pd.DataFrame(test_qs)

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df.text, shuffle=False, test_size=.20, random_state=seed)

In [0]:
# create tokenizer classs 
from tensorflow.keras.preprocessing import text

class TextPreprocessor(object):
  def __init__(self, vocab_size):
    self._vocab_size = vocab_size
    self._tokenizer = None

  def create_tokenizer(self,text_list):
    tokenizer = text.Tokenizer(num_words=self._vocab_size)
    tokenizer.fit_on_texts(text_list)
    self._tokenizer = tokenizer

  def transform_text(self, text_list):
    text_matrix = self._tokenizer.texts_to_matrix(text_list)
    return text_matrix 

In [0]:
# import TextPreprocessor
vocab_size = 400

tokenizer = TextPreprocessor(vocab_size)
tokenizer.create_tokenizer(train_df)

bag_of_words_train = tokenizer.transform_text(train_df)
bag_of_words_test = tokenizer.transform_text(test_df)

In [0]:
print(bag_of_words_train)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [0]:
import pickle

with open('./processor_state.pkl', 'wb') as f:
  pickle.dump(tokenizer, f)

In [0]:
tags_split = [tags.split(',') for tags in df['extracted_tags'].values]

In [0]:
tags_split

[['pandas'],
 ['matplotlib'],
 ['matplotlib'],
 ['scikitlearn'],
 ['matplotlib'],
 ['pandas'],
 ['keras'],
 ['pandas', 'scikitlearn'],
 ['pandas'],
 ['pandas'],
 ['matplotlib'],
 ['scikitlearn'],
 ['pandas'],
 ['tensorflow'],
 ['tensorflow', 'keras'],
 ['matplotlib'],
 ['pandas'],
 ['tensorflow'],
 ['pandas'],
 ['matplotlib'],
 ['pandas', 'matplotlib'],
 ['tensorflow'],
 ['pandas'],
 ['pandas', 'matplotlib'],
 ['pandas'],
 ['matplotlib'],
 ['pandas'],
 ['matplotlib'],
 ['pandas'],
 ['tensorflow', 'keras'],
 ['pandas'],
 ['pandas'],
 ['matplotlib'],
 ['pandas'],
 ['pandas'],
 ['pandas'],
 ['pandas'],
 ['tensorflow'],
 ['pandas'],
 ['keras'],
 ['matplotlib'],
 ['pandas'],
 ['pandas'],
 ['keras'],
 ['pandas'],
 ['scikitlearn'],
 ['pandas'],
 ['matplotlib'],
 ['matplotlib'],
 ['pandas'],
 ['tensorflow'],
 ['tensorflow'],
 ['pandas'],
 ['tensorflow'],
 ['tensorflow'],
 ['pandas'],
 ['matplotlib'],
 ['pandas'],
 ['pandas', 'matplotlib'],
 ['pandas'],
 ['matplotlib'],
 ['matplotlib'],
 ['pand

In [0]:
tag_encoder = MultiLabelBinarizer()
tags_encoded = tag_encoder.fit_transform(tags_split)

In [0]:
train_tags, test_tags = train_test_split(tags_encoded, test_size=.20, shuffle=False, random_state = seed)


In [0]:
tag_encoder.classes_
num_tags = 5

In [0]:
def create_model(vocab_size, num_tags):
  model = Sequential()  
  model.add(tf.keras.layers.Dense(50, input_shape=(vocab_size, ), activation='relu'))
  model.add(tf.keras.layers.Dense(25, activation='relu'))
  model.add(tf.keras.layers.Dense(num_tags, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model(vocab_size, num_tags)
model.summary()

W0816 20:44:25.734153 140423375075200 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0816 20:44:25.845103 140423375075200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                20050     
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 130       
Total params: 21,455
Trainable params: 21,455
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(bag_of_words_train, train_tags, epochs=3, batch_size = 128, validation_split=0.1)
model.evaluate(bag_of_words_test, test_tags, batch_size=128 )


Train on 135503 samples, validate on 15056 samples
Epoch 1/3
135503/135503 [==============================] - 5s 35us/sample - loss: 0.1450 - acc: 0.9441 - val_loss: 0.1093 - val_acc: 0.9586
Epoch 2/3
135503/135503 [==============================] - 5s 34us/sample - loss: 0.1056 - acc: 0.9590 - val_loss: 0.1023 - val_acc: 0.9603
Epoch 3/3
37640/37640 [==============================] - 1s 17us/sample - loss: 0.1005 - acc: 0.9606


[0.10051474966797641, 0.9605845]

In [0]:
model.save('keras_saved_model.h5')

In [0]:
# use custom model prediction class 

In [0]:
%%writefile model_prediction.py
import pickle
import os
import numpy as np

class Custom_Model_prediction(object):
  def __init__(self, model, processor):
    self._model = model 
    self._processor = processor 

  def predict(self, instances, **kwargs):
    preprocessed_data = self._processor.transform_text(instances)
    predictions = self._model.predict(preprocessed_data)
    return predictions.tolist()
    
  @classmethod
  def from_path(cls, model_dir):
    import tensorflow.keras as keras
    model = keras.models.load_model(
        os.path.join(model_dir, 'keras_saved_model.h5'))
    with open(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
      processor = pickle.load(f)

    return cls(model, processor)


Overwriting model_prediction.py


In [0]:
test_requests = [
                 "How to preprocess strings in Keras models Lambda layer? I have the problem that the value passed on to the Lambda layer (at compile time) is a placeholder generated by keras (without values). When the model is compiled, the .eval () method throws the error: You must feed a value for placeholder tensor 'input_1' with dtype string and shape [?, 1] def text_preprocess(x): strings = tf.keras.backend.eval(x) vectors = [] for string in strings: vector = string_to_one_hot(string.decode('utf-8')) vectors.append(vector) vectorTensor = tf.constant(np.array(vectors),dtype=tf.float32) return vectorTensor input_text = Input(shape=(1,), dtype=tf.string) embedding = Lambda(text_preprocess)(input_text) dense = Dense(256, activation='relu')(embedding) outputs = Dense(2, activation='softmax')(dense) model = Model(inputs=[input_text], outputs=outputs) model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy']) model.summary() model.save('test.h5') If I pass a string array into the input layer statically, I can compile the model, but I get the same error if I want to convert the model to tflite. #I replaced this line: input_text = Input(shape=(1,), dtype=tf.string) #by this lines: test = tf.constant(['Hello', 'World']) input_text = Input(shape=(1,), dtype=tf.string, tensor=test) #but calling this ... converter = TFLiteConverter.from_keras_model_file('string_test.h5') tfmodel = converter.convert() #... still leads to this error: InvalidArgumentError: You must feed a value for placeholder tensor 'input_3' with dtype string and shape [2] [[{{node input_3}}]] ",
  "Change the bar item name in Pandas I have a test excel file like: df = pd.DataFrame({'name':list('abcdefg'), 'age':[10,20,5,23,58,4,6]}) print (df) name  age 0    a   10 1    b   20 2    c    5 3    d   23 4    e   58 5    f    4 6    g    6 I use Pandas and matplotlib to read and plot it: import pandas as pd import numpy as np import matplotlib.pyplot as plt import os excel_file = 'test.xlsx' df = pd.read_excel(excel_file, sheet_name=0) df.plot(kind='bar') plt.show() the result shows: enter image description here it use index number as item name, how can I change it to the name, which stored in column name?"
]

In [0]:
from model_prediction import Custom_Model_prediction 

classifier = Custom_Model_prediction.from_path('.')
results = classifier.predict(test_requests)
print(results)

for i in range(len(results)):
  print('Predicted labels:')
  for idx,val in enumerate(results[i]):
    if val > 0.7:
      print(tag_encoder.classes_[idx])
  print('\n')



W0816 20:44:41.417402 140423375075200 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0816 20:44:41.419111 140423375075200 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[[0.9440398216247559, 4.470348358154297e-07, 0.0005476176738739014, 0.0009645521640777588, 0.6134801506996155], [0.00012993812561035156, 0.7955620288848877, 0.7841851711273193, 0.0010597561486065388, 5.078385220258497e-05]]
Predicted labels:
keras


Predicted labels:
matplotlib
pandas




In [0]:
# package our model and deploy to AI platform 

%%writefile setup.py

from setuptools import setup 

setup(
    name = "so_predict", 
    version = '0.1',
    include_package_data = True,
    scripts = ['preprocess.py', 'model_prediction.py']
)

Writing setup.py


In [0]:
## replace this with the name of your cloud storage bucket 

!gsutil cp keras_saved_model.h5 gs://your_gcs_bucket/
!gsutil cp processor_state.pkl gs://your_gcs_bucket/

Copying file://keras_saved_model.h5 [Content-Type=application/octet-stream]...
You are attempting to access protected data with no configured
credentials. Please visit https://cloud.google.com/console#/project
and sign up for an account, and then run the "gcloud auth login"
command to configure gsutil to use these credentials.
Copying file://processor_state.pkl [Content-Type=application/octet-stream]...
ResumableUploadAbortException: 403 The project to be billed is associated with a closed billing account.
